In [ ]:
from keras.datasets import mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [ ]:
from keras import Sequential, layers
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), padding='Same', input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(LeakyReLU())

model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), padding='Same'))
model.add(layers.BatchNormalization())
model.add(LeakyReLU())

model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding='Same'))
model.add(layers.BatchNormalization())
model.add(LeakyReLU())

model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), padding='Same'))
model.add(layers.BatchNormalization())
model.add(LeakyReLU())


model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.BatchNormalization())
model.add(LeakyReLU())
model.add(layers.Dense(10, activation="softmax"))

model.summary()

In [ ]:
from keras import optimizers

optimizer = optimizers.Nadam()

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
from keras import callbacks

learning_rate_reduction = callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

epochs = 30 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_split=0.2,
                              verbose = 2, batch_size=batch_size
                              , callbacks=[learning_rate_reduction])